# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [3]:
import pandas as pd

# import the train dataset ( 1000 first rows)

csv_file = "../../../data/train.csv"

df = pd.read_csv(csv_file, nrows=1000)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
995,2011-07-30 07:33:13.0000001,16.9,2011-07-30 07:33:13 UTC,-73.884801,40.755707,-73.980472,40.765556,1
996,2011-09-24 23:20:00.000000112,13.7,2011-09-24 23:20:00 UTC,-73.953603,40.779203,-73.995763,40.726701,3
997,2011-12-22 11:07:00.00000037,29.3,2011-12-22 11:07:00 UTC,-73.942380,40.837712,-73.864372,40.769985,2
998,2014-11-03 12:40:00.00000014,6.5,2014-11-03 12:40:00 UTC,-73.961151,40.774578,-73.972251,40.785640,1


In [4]:
# prepare X and y
y = df['fare_amount']
X = df.drop(columns=['fare_amount'])
X

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...
995,2011-07-30 07:33:13.0000001,2011-07-30 07:33:13 UTC,-73.884801,40.755707,-73.980472,40.765556,1
996,2011-09-24 23:20:00.000000112,2011-09-24 23:20:00 UTC,-73.953603,40.779203,-73.995763,40.726701,3
997,2011-12-22 11:07:00.00000037,2011-12-22 11:07:00 UTC,-73.942380,40.837712,-73.864372,40.769985,2
998,2014-11-03 12:40:00.00000014,2014-11-03 12:40:00 UTC,-73.961151,40.774578,-73.972251,40.785640,1


In [5]:
from sklearn.model_selection import train_test_split
# Hold out ( train and test dplit )
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state=42)

In [6]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
82,2013-06-14 08:27:43.0000001,2013-06-14 08:27:43 UTC,-73.953710,40.790813,-73.957015,40.777676,1
991,2011-10-05 20:50:00.000000227,2011-10-05 20:50:00 UTC,-73.961407,40.765273,-73.954393,40.784030,2
789,2015-01-10 21:18:25.0000001,2015-01-10 21:18:25 UTC,-73.984497,40.728798,-73.984085,40.725384,1
894,2009-06-21 18:56:00.000000158,2009-06-21 18:56:00 UTC,-74.004737,40.751987,-73.974168,40.743227,5
398,2015-03-11 14:48:58.0000006,2015-03-11 14:48:58 UTC,-73.973305,40.756386,-73.872444,40.774307,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [7]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [8]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        # A COMPPLETER
        self.start_lat=start_lat
        self.start_lon=start_lon
        self.end_lat=end_lat
        self.end_lon=end_lon
    
    def fit(self, X, y=None):
        # A COMPLETER 
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_['distance'] = haversine_vectorized(X_, self.start_lat, self.start_lon, self.end_lat, self.end_lon)
        return X_[['distance']]

In [9]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
82,1.487035
991,2.167697
789,0.381221
894,2.753213
398,8.724895


In [10]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self, datetime_column ):
       # A COMPLETER
        self.datetime_column = datetime_column
        
    def fit(self, X, y=None):
        # A COMPLETER
        return self

    def transform(self, X, y=None):
        # A COMPLETER
        X_ = X.copy()
        X_[self.datetime_column] = pd.to_datetime(X_[self.datetime_column])
        X_['dow'] = X_[self.datetime_column].dt.dayofweek
        X_['hour'] = X_[self.datetime_column].dt.hour
        X_['month'] = X_[self.datetime_column].dt.month
        X_['year'] = X_[self.datetime_column].dt.year
        return X_[['dow', 'hour', 'month', 'year']]

In [11]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
82,4,8,6,2013
991,2,20,10,2011
789,5,21,1,2015
894,6,18,6,2009
398,2,14,3,2015


###  Prepocessing pipeline

In [12]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [13]:
# create distance pipeline dist_pipe
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

dist_pipe = Pipeline([
    ('dist_trans', DistanceTransformer()),
    ('stdscaler', StandardScaler())
])

# display distance pipeline
dist_pipe

Pipeline(steps=[('dist_trans', DistanceTransformer()),
                ('stdscaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [14]:
from sklearn.preprocessing import OneHotEncoder

# create time pipeline time_pipe
time_pipe = Pipeline([
    ('time_enc', TimeFeaturesEncoder('pickup_datetime')),
    ("ohe", OneHotEncoder(handle_unknown='ignore'))
])

# display time pipeline
time_pipe

Pipeline(steps=[('time_enc',
                 TimeFeaturesEncoder(datetime_column='pickup_datetime')),
                ('ohe', OneHotEncoder(handle_unknown='ignore'))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

**Note pour moi-même** :

The ColumnTransformer allows you to apply different transformers to different subsets of the columns in your data. 

The ColumnTransformer takes as input a list of transformers, where each transformer is specified as a tuple containing the name of the transformer, the estimator (pipeline), and the list of columns to be transformed by that estimator

In [15]:
from sklearn.compose import ColumnTransformer

# create preprocessing pipeline preproc_pipe
preproc_pipe = ColumnTransformer(
    transformers=[        
        ('distance', dist_pipe, ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']),
        ('time', time_pipe, ['pickup_datetime'])
    ]
)
# display preprocessing pipeline
preproc_pipe

ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('dist_trans',
                                                  DistanceTransformer()),
                                                 ('stdscaler',
                                                  StandardScaler())]),
                                 ['pickup_latitude', 'pickup_longitude',
                                  'dropoff_latitude', 'dropoff_longitude']),
                                ('time',
                                 Pipeline(steps=[('time_enc',
                                                  TimeFeaturesEncoder(datetime_column='pickup_datetime')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [16]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
pipe = Pipeline([
    ('features', ColumnTransformer(transformers=[
        ('distance', dist_pipe, ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']),
        ('time', time_pipe, ['pickup_datetime'])
    ])),
    ('model', LinearRegression())
])
# display the pipeline with model
pipe

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_enc',
                                                                   TimeFeaturesEncoder(datetime_column='pickup_datetime')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('model', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [17]:
# train the pipelined model
pipe.fit(X_train, y_train)

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_latitude',
                                                   'pickup_longitude',
                                                   'dropoff_latitude',
                                                   'dropoff_longitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_enc',
                                                                   TimeFeaturesEncoder(datetime_column='pickup_datetime')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pickup_datetime'])])),
                ('model', LinearRegression())])

In [18]:
# compute y_pred on the test set
y_pred = pipe.predict(X_test)

Use the RMSE to evaluate the performance of the model:

In [19]:
from sklearn.metrics import mean_squared_error

def compute_rmse(y_pred, y_true):
    return mean_squared_error(y_true, y_pred, squared=False)

In [20]:
# call compute_rmse
compute_rmse(y_pred, y_test)

9.269221737266113

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [28]:
def get_data(nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    csv_file = "../../../data/train.csv"
    df = pd.read_csv(csv_file, nrows=nrows)
    return df

In [29]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    df = df.query("fare_amount > 0 & passenger_count <= 8 & passenger_count > 0")
    return df

In [30]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
    dist_pipe = Pipeline([
        ('dist_trans', DistanceTransformer()),
        ('stdscaler', StandardScaler())
    ])
    
    time_pipe = Pipeline([
        ('time_enc', TimeFeaturesEncoder('pickup_datetime')),
        ("ohe", OneHotEncoder(handle_unknown='ignore'))
    ])
    
    pipe = Pipeline([
    ('features', ColumnTransformer(transformers=[
        ('distance', dist_pipe, ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']),
        ('time', time_pipe, ['pickup_datetime'])
    ])),
    ('model', LinearRegression())
    ])
    
    return pipe

In [31]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
   # A COMPLETER
    return pipeline.fit(X_train, y_train)

In [32]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    # A COMPLETER
    y_pred = pipe.predict(X_test)
    rmse = compute_rmse(y_pred, y_test)
    print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [33]:
# store the data in a DataFrame
df = get_data()
df = clean_data(df)
# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

9.796375276588062


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀